### 1) Run this cell to install necessary packages

In [ ]:
!pip install pdfmerge

### 2) Setup
Go into your Google Drive and create a folder on the homepage called "PDFmerge". Upload the PDFs you want to merge into this folder. Then run this cell and make sure to give it access to your Drive. When the running of this cell is complete, all the files in the PDFmerge folder should be listed in the output.

In [ ]:
import IPython
from IPython.display import display, Javascript
from pdfmerge import pdfmerge
from google.colab import drive
from google.colab import output
import os

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/PDFmerge
!ls /content/drive/MyDrive/PDFmerge

# List files in the specified directory
files = os.listdir("/content/drive/MyDrive/PDFmerge")
files_save_copy = files.copy()


### 3) Merge order
Run this cell to change the order in which the PDFs should be merged. The output will create a drag and drop menu for easy modifying. After that, press "Save". Feel free to run this cell multiple times if you're not happy with the order.

In [ ]:
# Function to generate HTML for the list items
def generate_html(pdf_list):
    html_code = """
    <ul id="draggable-list" style="list-style-type: none; padding: 0; max-width: 200px">
    """
    for item in pdf_list:
        html_code += f'<li draggable="true" ondragstart="drag(event)" id="{item}" style="padding: 5px; border: 1px solid #ddd; margin: 5px; background-color: #434343; cursor: move;">{item}</li>'
    html_code += """
    </ul>
    <button id="save-button" style="margin-top: 10px;">Save</button>
    <script>
    var dragged;

    document.addEventListener("drag", function(event) {}, false);

    document.addEventListener("dragstart", function(event) {
      dragged = event.target;
      event.target.style.opacity = 0.5;
    }, false);

    document.addEventListener("dragend", function(event) {
      event.target.style.opacity = "";
    }, false);

    document.addEventListener("dragover", function(event) {
      event.preventDefault();
    }, false);

    document.addEventListener("dragenter", function(event) {
      if (event.target.tagName == "LI") {
        event.target.style.background = "#e0e0e0";
      }
    }, false);

    document.addEventListener("dragleave", function(event) {
      if (event.target.tagName == "LI") {
        event.target.style.background = "";
      }
    }, false);

    document.addEventListener("drop", function(event) {
      event.preventDefault();
      if (event.target.tagName == "LI") {
        event.target.style.background = "";
        dragged.parentNode.removeChild(dragged);
        event.target.parentNode.insertBefore(dragged, event.target.nextSibling);
      }

      // Save the new order silently
      var listItems = document.getElementById('draggable-list').children;
      var newOrder = [];
      for (var i = 0; i < listItems.length; i++) {
        newOrder.push(listItems[i].id);
      }
      google.colab.kernel.invokeFunction('notebook.SaveOrderSilently', [newOrder], {});
    }, false);

    document.getElementById('save-button').addEventListener('click', function() {
      // Save the new order and disable drag and drop
      var listItems = document.getElementById('draggable-list').children;
      var newOrder = [];
      for (var i = 0; i < listItems.length; i++) {
        newOrder.push(listItems[i].id);
      }
      google.colab.kernel.invokeFunction('notebook.SaveOrderAndPrint', [newOrder], {});

      // Disable drag and drop
      var list = document.getElementById('draggable-list');
      for (var i = 0; i < list.children.length; i++) {
        list.children[i].setAttribute('draggable', 'false');
        list.children[i].style.cursor = 'default';
      }
      document.getElementById('save-button').disabled = true;
    });
    </script>
    """
    return html_code

# Function to save the new order of items silently
def save_order_silently(new_order):
    global files
    files = new_order
    for filename in files_save_copy:
        if filename not in files:
            files.append(filename)

# Function to save the new order of items and print the new order
def save_order_and_print(new_order):
    global files
    files = new_order
    for filename in files_save_copy:
        if filename not in files:
            files.append(filename)
    output.clear()
    print("File merge order:\n")
    for i, filename in enumerate(files):
      print((str(i+1) +")").rjust(5, " "), filename)


# Register the functions to be called from JavaScript
from google.colab import output
output.register_callback('notebook.SaveOrderSilently', save_order_silently)
output.register_callback('notebook.SaveOrderAndPrint', save_order_and_print)

# Display the draggable list
html_code = generate_html(files)
display(IPython.display.HTML(html_code))


### 4) Merge the files

Run the code to enter the file name of the merged PDFs. Press enter and the merging process will start. The merged file can be found in the PDFmerge folder under the name you entered.

In [ ]:
merged_filename = str(input())
pdfmerge(files, merged_filename + ".pdf")